In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
print(tf.__version__)

/Users/sreeharirammohan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


In [4]:
X = np.load('/Volumes/Elements/CES-Model_and_Data/X.npy')
y = np.load('/Volumes/Elements/CES-Model_and_Data/y.npy')

In [5]:
#Read in images from folder
DATA_PATH = "/Volumes/Elements/Lesions"

classes = os.listdir(DATA_PATH)
del classes[0] #first entry is .DS Store not a directory

print(classes)

X = np.array([])
y = np.array([])

for class_name in classes:
    
    print("Working on class", class_name)

    class_folder = os.path.join(DATA_PATH,class_name)

    sub_files = os.listdir(class_folder)
    
    number_of_files = len(sub_files)
    i = 0
    
    for file in sub_files:
        if '.jpg' in file:
            image_filepath = os.path.join(class_folder, file)
            image_as_np_array = cv2.imread(image_filepath,flags=cv2.IMREAD_COLOR)
            
            #decrease the size of the image by 1/4 to 
            image_as_np_array = cv2.resize(image_as_np_array, (56,75), interpolation = cv2.INTER_LINEAR)
            
            #cv2.imshow('ImageWindow', image_as_np_array)
            #cv2.waitKey()
            
            #print(image_as_np_array.shape)

            X = np.append(X, image_as_np_array)
            y = np.append(y, class_name)

            print(str(i) + " - " + str(round((i/number_of_files)*100, 2)) + "%", end='\r', flush=True)

            i += 1
        


['Actinic_Keratosis', 'Basel_Cell_Carcinoma', 'Dermatofibroma', 'Melanoma', 'Nevus', 'Seaborrheic Keratosis']
Working on class Actinic_Keratosis
Working on class Basel_Cell_Carcinoma
Working on class Dermatofibroma
Working on class Melanoma
Working on class Nevus
Working on class Seaborrheic Keratosis


In [6]:
X = X.reshape(-1,56,75,3)
X.shape

(2093, 56, 75, 3)

In [7]:
y.shape

(2093,)

In [8]:
#Normalize image array values between 0-1
X /= 255.0

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [10]:
print(y)

[0 0 0 ... 5 5 5]


In [11]:
np.save('/Volumes/Elements/CES-Model_and_Data/X.npy', X)
np.save('/Volumes/Elements/CES-Model_and_Data/y.npy', y)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
X_train.shape

(1402, 56, 75, 3)

In [14]:
X_test.shape

(691, 56, 75, 3)

In [15]:
y_train.shape

(1402,)

In [16]:
y_test.shape

(691,)

In [17]:
print(y_test)

[5 3 1 0 0 5 1 4 5 5 1 3 0 4 1 5 0 1 0 4 4 1 3 0 3 4 2 0 0 1 1 5 3 0 5 3 5
 3 1 4 2 1 1 1 1 5 2 1 3 0 1 1 1 0 1 3 3 1 1 0 3 5 3 1 4 1 0 1 3 4 0 1 1 5
 4 5 0 5 0 0 1 0 0 1 3 1 4 5 1 4 5 3 1 1 3 4 3 4 1 4 0 4 5 4 5 5 5 0 1 1 3
 4 4 0 5 1 0 2 5 1 4 4 3 1 0 5 5 1 5 3 0 1 4 5 5 4 1 2 1 1 1 1 0 0 0 1 5 0
 0 0 4 1 3 1 1 1 4 3 4 4 4 5 4 4 4 0 1 5 4 3 4 2 1 1 1 4 1 1 4 3 3 3 2 4 5
 5 4 0 3 0 1 1 4 1 1 1 4 3 0 0 1 3 3 1 3 2 0 4 3 0 3 1 1 0 1 2 4 5 4 3 4 1
 4 0 1 2 4 1 5 1 3 4 5 1 5 1 5 3 2 1 5 1 5 5 5 1 0 1 4 5 1 3 3 4 5 0 5 1 3
 5 5 5 1 5 5 4 5 5 0 5 1 1 5 0 1 5 1 1 5 0 2 5 3 4 5 3 0 1 4 4 3 1 2 1 4 5
 5 2 1 5 0 1 0 0 0 3 1 2 5 5 5 0 3 5 1 1 1 3 5 5 4 2 2 5 0 3 1 1 3 0 1 3 1
 5 1 4 1 3 4 4 5 5 0 4 4 4 5 5 0 4 0 4 0 4 3 1 1 0 0 5 2 1 4 0 1 4 5 0 5 1
 4 0 1 5 1 2 5 1 3 5 0 1 4 1 0 3 0 4 1 1 3 3 4 1 4 1 1 0 5 4 0 1 0 0 4 3 1
 4 5 1 5 1 5 0 3 4 3 4 4 3 5 3 5 5 1 5 5 1 0 4 4 0 0 4 4 1 0 1 4 3 3 0 2 1
 4 5 1 3 3 3 0 3 1 5 3 0 1 3 3 4 5 3 3 0 4 1 1 3 0 2 4 1 5 1 0 1 1 4 1 4 4
 0 1 5 1 1 4 1 0 5 1 1 5 

In [32]:
import keras 
from keras.models import Sequential
from keras.layers import *

num_classes = 4
input_shape=(56,75,3)
epochs = 100
batch_size = 32

In [33]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [34]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          verbose=1)

Train on 973 samples, validate on 480 samples
Epoch 1/50
973/973 [==============================] - 190s 195ms/step - loss: 1.2956 - acc: 0.3803 - val_loss: 1.2506 - val_acc: 0.4208
Epoch 2/50
973/973 [==============================] - 189s 194ms/step - loss: 1.2762 - acc: 0.3864 - val_loss: 1.2681 - val_acc: 0.4188
Epoch 3/50
973/973 [==============================] - 186s 191ms/step - loss: 1.2575 - acc: 0.4029 - val_loss: 1.2926 - val_acc: 0.4208
Epoch 4/50
973/973 [==============================] - 185s 190ms/step - loss: 1.2629 - acc: 0.3936 - val_loss: 1.2264 - val_acc: 0.4208
Epoch 5/50
973/973 [==============================] - 185s 190ms/step - loss: 1.2502 - acc: 0.4121 - val_loss: 1.2114 - val_acc: 0.4562
Epoch 6/50
973/973 [==============================] - 189s 194ms/step - loss: 1.2349 - acc: 0.4286 - val_loss: 1.4935 - val_acc: 0.2875
Epoch 7/50
973/973 [==============================] - 204s 210ms/step - loss: 1.2319 - acc: 0.4573 - val_loss: 1.3139 - val_acc: 0.3000
Ep

In [37]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8730818072954813
Test accuracy: 0.6520833333333333


In [39]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    tf.keras.models.save_model(
        model,
        '/Volumes/Elements/Lesions/65%_model_10-9.h5',
        overwrite=True,
        include_optimizer=True
    )

In [40]:
converter = tf.contrib.lite.TocoConverter.from_keras_model_file('/Volumes/Elements/Lesions/65%_model_10-9.h5')
converter.post_training_quantize = True
converter.quantize_weights = True
tflite_model = converter.convert()
open("/Volumes/Elements/Lesions/65%_epoch_model_10-9_quantized_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


10624816

In [41]:
print("done")

done
